# Output Results to GeoTiff

*Author: Alex Lewandowski; Alaska Satellite Facility*

Output the results of your MintPy Time Series to GeoTiff



In [16]:
from ipyfilechooser import FileChooser
from pathlib import Path

import h5py
import rasterio

---
## 1. Select your project's custom config file

- This is located in your project's `MintPy` directory
- It is a text file named after your project
  - `path/to/MinPy/my_project.txt`

In [2]:
path = Path.home()
fc = FileChooser(path)
print("Select your project's MintPy directory:")
display(fc)

Select your project's MintPy directory:


FileChooser(path='/home/jovyan', filename='', title='', show_hidden=False, select_desc='Select', change_desc='…

In [12]:
config_path = Path(fc.selected)
mint_path = config_path.parent
inputs_path = mint_path/ 'inputs'
ifgramstack = inputs_path / 'ifgramStack.h5'
geotiff_path = mint_path/'GeoTiffs'
ts_demErr_path = list(mint_path.glob('timeseries*_demErr.h5'))[0]

**Create a list dates for all timesteps**

In [8]:
ifgramstack = inputs_path/"ifgramStack.h5"

with h5py.File(ifgramstack, "r") as f:
    dates = f["date"][()]
    dates = list(set([d.decode("utf-8") for insar in dates for d in insar]))
    dates.sort()
dates

['20230622',
 '20230704',
 '20230716',
 '20230728',
 '20230809',
 '20230821',
 '20230902',
 '20230914',
 '20230926',
 '20231008',
 '20231020',
 '20231101',
 '20231113',
 '20231125',
 '20231207',
 '20231219',
 '20231231',
 '20240112',
 '20240124',
 '20240205',
 '20240217',
 '20240229',
 '20240312']

**Save the full displacement timeseries**

In [14]:
ds = f'{dates[0]}_{dates[-1]}'
!save_gdal.py $ts_demErr_path -d $ds --of GTIFF -o $geotiff_path/"save_gdal_ts_demErr.tif"

read 20240312 from file: /home/jovyan/AZ_burst_2/MintPy/timeseries_demErr.h5
read 20230622 from file: /home/jovyan/AZ_burst_2/MintPy/timeseries_demErr.h5
initiate GDAL driver: GeoTIFF
create raster band:
  raster row / column number: 498, 1201
  raster data type: 6 (float32)
set transform info: (294560.0, 80.0, 0, 3727840.0, 0, -80.0)
set projection as: EPSG 32612
write data to raster band
finished writing to /home/jovyan/AZ_burst_2/MintPy/GeoTiffs/save_gdal_ts_demErr.tif


**Save the unwrapped displacement GeoTiffs**

In [ ]:
with h5py.File(ifgramstack, 'r') as f:
    unw_pth = f.attrs['FILE_PATH']

ds_unw = rasterio.open(unw_pth, 'r', driver='GTiff')

for i, d in enumerate(tqdm(dates)):
    date_range = f'{dates[0]}_{dates[i]}'
    cmd = f'view.py {ts_demErr} {date_range} --notitle --notick --noaxis'
    data, _, _ = mintpy.view.prep_slice(cmd)
    data = data / 100 # cm -> meters
        

    with rasterio.open(f'{unwrapped_path}/{date_range}_{ts_demErr.stem}_unwrapped.tif', 'w', driver='GTiff',
                  height = data.shape[0], width = data.shape[1],
                  count=1, dtype=str(data.dtype),
                  crs=ds_unw.read_crs(),
                  transform=ds_unw.transform,
                  nodata=np.nan) as ds:
        
        ds.write(data.astype(rasterio.float32), 1)